In [ ]:
pip install python-chess

In [ ]:
import pandas as pd
import numpy as np
import chess.pgn
import chess
from random import randint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, losses

In [ ]:
def fen2bitstring(fen):
    board = chess.Board(fen)
    bit_string = np.zeros(773, dtype = np.int8)
    
    def piece_index(piece_type):
        if piece_type == chess.PAWN:
            return 0
        elif piece_type == chess.KNIGHT:
            return 1
        elif piece_type == chess.BISHOP:
            return 2
        elif piece_type == chess.ROOK:
            return 3
        elif piece_type == chess.QUEEN:
            return 4
        elif piece_type == chess.KING:
            return 5
        else:
            return -1
    
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            bit_index = 2 * 6 * square
            bit_string[bit_index] = int(piece.color)  #side (White: 1, Black: 0)
            bit_index += 1 + 6 * piece_index(piece)  #piece type (pawn: 0, knight: 1, bishop: 2, rook: 3, queen: 4, king: 5)
            bit_string[bit_index] = 1
            #We have encode 768 bits
    bit_string[768] = int(board.turn)
    bit_string[769] = int(board.has_kingside_castling_rights(chess.WHITE))
    bit_string[770] = int(board.has_queenside_castling_rights(chess.WHITE))
    bit_string[771] = int(board.has_kingside_castling_rights(chess.BLACK))
    bit_string[772] = int(board.has_queenside_castling_rights(chess.BLACK))
    
    return bit_string

In [ ]:
def randPosiGeneratorFromGame(game):
    random_positions_array = []
    positions = []
    board = game.board()

    for move in game.mainline_moves():
        if not board.is_capture(move):
            position = board.fen()
            bitstring_position = fen2bitstring(position)
            positions.append(bitstring_position)

        board.push(move)

        no_capture_posis = len(positions)
    if no_capture_posis > 5:
        for i in range(10):
            random_index = randint(5, no_capture_posis - 1)
            random_positions_array.append(positions[random_index])


    return random_positions_array

In [ ]:
pgn_file = open("../input/ccrl-dataset/CCRLdb.pgn")

random_positions_white_win_bitstring = []
random_positions_black_win_bitstring = []

i = 0
while i < 100000:
    game = chess.pgn.read_game(pgn_file)
    if game.headers["Result"] == "1-0":
        temp1 = randPosiGeneratorFromGame(game)
        random_positions_white_win_bitstring += temp1
    elif game.headers["Result"] == "0-1":
        temp2 = randPosiGeneratorFromGame(game)
        random_positions_black_win_bitstring += temp2
    if game is None:
        break
    i = i + 1
        
pgn_file.close()

In [ ]:
len(random_positions_white_win_bitstring)

In [ ]:
white_win_bitstring_tensor = tf.convert_to_tensor(random_positions_white_win_bitstring, dtype=tf.float32)
black_win_bitstring_tensor = tf.convert_to_tensor(random_positions_black_win_bitstring, dtype=tf.float32)

In [ ]:
white_win_dataset = tf.data.Dataset.from_tensor_slices(white_win_bitstring_tensor)
black_win_dataset = tf.data.Dataset.from_tensor_slices(black_win_bitstring_tensor)

In [ ]:
concatenated_dataset = white_win_dataset.concatenate(black_win_dataset)

In [ ]:
num_samples = len(concatenated_dataset)
train_dataset = concatenated_dataset.shuffle(num_samples)
print('Train Dataset Size:', len(train_dataset))

In [ ]:
def scheduler(epoch, lr):
    return lr * 0.98

# Create the learning rate scheduler callback
lr_scheduler = callbacks.LearningRateScheduler(scheduler)

# Set the initial learning rate
initial_learning_rate = 0.005

In [ ]:
train_data = np.array(list(train_dataset.as_numpy_iterator()))

In [ ]:
def train_autoencoders(train_data):  #[[e1, d1], [c1], [e2, d2], [c2], [e3, d3], [c3], [e4, d4], [o]]
    weights = []
    # Layer 1: 773-600-773 Autoencoder
    layer1 = tf.keras.Sequential()
    encoder1 = layers.Dense(600, activation='relu', input_shape=(773,))
    layer1.add(encoder1)
    decoder1 = layers.Dense(773, activation='relu')
    layer1.add(decoder1)
    layer1.compile(optimizer=tf.keras.optimizers.Adamax(), loss = losses.MeanSquaredError())
    layer1.fit(train_data, train_data, epochs=200, batch_size = 500, callbacks=[lr_scheduler])
    layer1_model = tf.keras.models.Model(inputs=layer1.input, outputs=layer1.layers[0].output)
    weights.append(layer1.layers[0].get_weights())
    
    intermediate_output1 = layer1_model.predict(train_data)
    
    # Layer 2: 600-400-600 Autoencoder
    layer2 = tf.keras.Sequential()
    encoder2 = layers.Dense(400, activation='relu', input_shape=(600,))
    layer2.add(encoder2)
    decoder2 = layers.Dense(600, activation='relu')
    layer2.add(decoder2)
    layer2.compile(optimizer=tf.keras.optimizers.Adamax(), loss = losses.MeanSquaredError())
    layer2.fit(intermediate_output1, intermediate_output1, epochs = 200, batch_size = 500, callbacks=[lr_scheduler])
    layer2_model = tf.keras.models.Model(inputs=layer2.input, outputs=layer2.layers[0].output)
    weights.append(layer2.layers[0].get_weights())
    
    intermediate_output2 = layer2_model.predict(intermediate_output1)

    # Layer 3: 400-200-400 Autoencoder
    layer3 = tf.keras.Sequential()
    encoder3 = layers.Dense(200, activation='relu', input_shape=(400,))
    layer3.add(encoder3)
    decoder3 = layers.Dense(400, activation='relu')
    layer3.add(decoder3)
    layer3.compile(optimizer=tf.keras.optimizers.Adamax(), loss = losses.MeanSquaredError())
    layer3.fit(intermediate_output2, intermediate_output2, epochs = 200, batch_size = 500, callbacks=[lr_scheduler])
    layer3_model = tf.keras.models.Model(inputs=layer3.input, outputs=layer3.layers[0].output)
    weights.append(layer3.layers[0].get_weights())
    
    intermediate_output3 = layer3_model.predict(intermediate_output2)

    # Layer 4: 200-100-200 Autoencoder
    layer4 = tf.keras.Sequential()
    encoder4 = layers.Dense(100, activation='relu', input_shape=(200,))
    layer4.add(encoder4)
    decoder4 = layers.Dense(200, activation='relu')
    layer4.add(decoder4)
    layer4.compile(optimizer=tf.keras.optimizers.Adamax(), loss = losses.MeanSquaredError())
    layer4.fit(intermediate_output3, intermediate_output3, epochs = 200, batch_size = 750, callbacks=[lr_scheduler])
    layer4_model = tf.keras.models.Model(inputs=layer4.input, outputs=layer4.layers[0].output)
    weights.append(layer4.layers[0].get_weights())
    
    return weights

In [ ]:
trained_pos2vec_weights = train_autoencoders(train_data)

In [ ]:
np.save("trained_pos2vec_weights_100000", trained_pos2vec_weights)